In [2]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2023-05-29 16:38:27 +07:00)


In [3]:
import numpy as np
from fairscale.experimental.tooling.layer_memory_tracker import LayerwiseMemoryTracker
from fastai.vision.all import *

time: 6.33 s (started: 2023-05-29 16:38:27 +07:00)


In [4]:
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

time: 0 ns (started: 2023-05-29 16:38:33 +07:00)


In [5]:
# Set the size of the image
img_size = (64, 64)

def create_random_data():
    # Generate a random NumPy array
    rand_arr = np.random.rand(*img_size, 3) * 255
    rand_arr = rand_arr.astype(np.uint8)
    return rand_arr

def get_model_file_size(model): 
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()

    model_size = param_size
    print('Model size: {} Byte'.format(model_size))

# Create your learner and load your model
model = load_learner("../../../../model/nvidia-t4/VGG16_bn_non_native_fp16_accum2.pkl")

get_model_file_size(model.model)

# Set the model to evaluation mode
model.eval()


# Get activation memory
tracker = LayerwiseMemoryTracker()
tracker.monitor(model)

x = create_random_data()
preds = model.predict(x)


tracker.stop()


time: 2.17 s (started: 2023-05-29 16:38:33 +07:00)


In [6]:
forward_act = sum(t.event.memory_activations for t in tracker.forward_traces) 
print(f"Activation Memory: {forward_act} Byte")

Activation Memory: 9480232 Byte
time: 0 ns (started: 2023-05-29 16:38:36 +07:00)
